In [1]:
seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf

from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
import pickle

#Scikit-Learn Importing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer
from sklearn.svm import SVR

#Import PyWavelets for WaveSmoothing
import pywt
import mad
from statsmodels import robust

#Import Keras
import keras
from keras.layers import Input, Dense, BatchNormalization, Dropout, LSTM, RepeatVector, TimeDistributed
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, Adadelta
from tqdm import tqdm
from keras import backend as K

#Import Gridsearch
from hypopt import GridSearch

#Import Capstone Functions
from Capstone_Functions import *
cf = CapstoneFunctions()

In [3]:
Monthly = pd.read_pickle('data/Monthly.pkl')
Monthly2 = pd.read_pickle('data/MonthlyReturns.pkl')

In [4]:
WOMonthly = Monthly.iloc[:,14:15].copy()
WOMonthly.tail()

,WOMonthly
date,
2020-03-02,-0.050345
2020-04-01,-0.177528
2020-05-01,0.103999
2020-06-01,0.080806
2020-07-01,-0.000094


In [5]:
FIPMonthly = Monthly2.iloc[:,16:17].copy()
FIPMonthly.head()

,FIPMonthly
date,
1990-12-03,0.034620
1991-01-01,0.022021
1991-02-01,0.013218
1991-03-01,0.089188
1991-04-01,0.031861


In [6]:
def Train_Test_Split2(Data1, Port, Window, Lookback, scaler): 

        """ 
        Import Monthly data with Date as Index. Select which portfolio to train test split. Set number of
        lookback months and whether to use MinMaxScaler.

        Example:
        HiXdata, HiYdata, HiXtrain, HiYtrain, HiXtest, HiYtest, HiPredictedX, HiForecastX, HiX, HiY = \
        Train_Test_Split(Monthly, 'Hi', 60, 12, scaler = False)

        """ 


        Data1 = Data1.values

        TrainSplit = int(Window*0.90)
        TestSplit = int(Window*0.1)

        #Iterate Through Data and Creat Numpy arrays with 12 Months of \ 
        #lagged data for X and 13th Month for Y
        tmpX=[]
        tmpY=[]
        for A in range(len(Data1)-Lookback):
            tmp=Data1[A:(A + Lookback),Port]
            tmpX.append(tmp)
            tmpY.append(Data1[(A + Lookback),Port])
        Xdata = np.array(tmpX)
        Ydata = np.array(tmpY)

        tmpForeX=[]
        for A in range(len(Data1)-Lookback):
            tmpFore=Data1[A+1:(A+1 + Lookback),Port]
            tmpForeX.append(tmpFore)
        XdataFore = np.array(tmpForeX)

        Ydata = Ydata.reshape(len(Ydata),1)

        Shape = int(Xdata.shape[0])
        Shape2 = int(XdataFore.shape[0])


        #Create Empty Numpy Arrays
        Xtrain=np.ones(shape=(Shape-Window,TrainSplit+TestSplit,12))
        Ytrain=np.ones(shape=(Shape-Window,TrainSplit+TestSplit,1))
        Xtest=np.ones(shape=(Shape-Window,TestSplit,Lookback))
        Ytest=np.ones(shape=(Shape-Window,TestSplit,1))
        PredictedX=np.ones(shape=(Shape-Window,1,Lookback))
        ForecastX=np.ones(shape=(Shape2-Window,1,Lookback))
        X = np.ones(shape=(Shape-Window,Window,Lookback))
        Y = np.ones(shape=(Shape-Window,Window,1))

        #Fill Numpy Arrays with data
        for B in range(Shape-Window):
            Xtrain[B,:,:] = Xdata[B:B+TrainSplit+TestSplit,:]
            Ytrain[B,:,:] = Ydata[B:B+TrainSplit+TestSplit]
            Xtest[B,:,:] = Xdata[B+TrainSplit:B+Window,:]
            Ytest[B,:,:] = Ydata[B+TrainSplit:B+Window]
            PredictedX[B,:,:] = Xdata[B+Window,:]
            ForecastX[B,:,:] = XdataFore[B+Window,:]
            X[B,:,:] = Xdata[B:B+Window,:]
            Y[B,:,:] = Ydata[B:B+Window]

            if scaler:
                sc = MinMaxScaler()
                Xtrain[B,:,:] = sc.fit_transform(Xtrain[B,:,:])
                Xtest[B,:,:] = sc.transform(Xtest[B,:,:])
                PredictedX[B,:,:] = sc.transform(PredictedX[B,:,:])
                X[B,:,:] = sc.transform(X[B,:,:])



        return Xdata, Ydata, Xtrain, Ytrain, Xtest, Ytest, PredictedX, ForecastX, X, Y

In [7]:
#Create Train_Test_Split for High and Low Momentum returns with a 60 Month Lookback
WOXdata, WOYdata, WOXtrain, WOYtrain, WOXtest, WOYtest, WOPredictedX, WOForecastX, WOX, WOY = \
Train_Test_Split2(WOMonthly, 0, 100, 12, scaler = False)

FIPXdata, FIPYdata, FIPXtrain, FIPYtrain, FIPXtest, FIPYtest, FIPPredictedX, FIPForecastX, FIPX, FIPY = \
Train_Test_Split2(FIPMonthly, 0, 100, 12, scaler = False)

# SVR

In [11]:
grid=np.power([10,10,10,10,10],np.linspace(-5,-1,5))
grid=np.append(grid, 5*grid)

cgrid=np.power([10,10,10,10,10,10],np.linspace(-2,3,6))
cgrid=np.append(cgrid, 5*cgrid)

rbf = ['rbf']
parameters = {'gamma': grid, 'C': cgrid, 'epsilon': grid, 'kernel':['rbf']}

In [12]:
def predictSVR2(Xtrain, Ytrain, PredictedX):
    #param_grid = [parameters]
    final_pred = np.zeros(shape=(244,1))
    for X in range(final_pred.shape[0]):
        gs = GridSearch(model = SVR(), param_grid = parameters)
        gs.fit(Xtrain[X], Ytrain[X].ravel(), scoring ='neg_mean_squared_error')
        final_pred[X] = gs.predict(PredictedX[X,:])
        print('Forecast',X+1,'of',244)
    return final_pred

In [13]:
Forecast_WOMonthly_SVR = predictSVR2(WOXtrain, WOYtrain, WOForecastX)

Forecast 1 of 244
Forecast 2 of 244
Forecast 3 of 244
Forecast 4 of 244
Forecast 5 of 244
Forecast 6 of 244
Forecast 7 of 244
Forecast 8 of 244
Forecast 9 of 244
Forecast 10 of 244
Forecast 11 of 244
Forecast 12 of 244
Forecast 13 of 244
Forecast 14 of 244
Forecast 15 of 244
Forecast 16 of 244
Forecast 17 of 244
Forecast 18 of 244
Forecast 19 of 244
Forecast 20 of 244
Forecast 21 of 244
Forecast 22 of 244
Forecast 23 of 244
Forecast 24 of 244
Forecast 25 of 244
Forecast 26 of 244
Forecast 27 of 244
Forecast 28 of 244
Forecast 29 of 244
Forecast 30 of 244
Forecast 31 of 244
Forecast 32 of 244
Forecast 33 of 244
Forecast 34 of 244
Forecast 35 of 244
Forecast 36 of 244
Forecast 37 of 244
Forecast 38 of 244
Forecast 39 of 244
Forecast 40 of 244
Forecast 41 of 244
Forecast 42 of 244
Forecast 43 of 244
Forecast 44 of 244
Forecast 45 of 244
Forecast 46 of 244
Forecast 47 of 244
Forecast 48 of 244
Forecast 49 of 244
Forecast 50 of 244
Forecast 51 of 244
Forecast 52 of 244
Forecast 53 of 244
Fo

In [14]:
SVR_WO_Forecast = pd.DataFrame(Forecast_WOMonthly_SVR)

In [15]:
SVR_WO_Forecast.to_pickle('data/SVR_WO_Forecast.pkl')

In [16]:
stop

NameError: name 'stop' is not defined